# Demo code to import to a new project from a zip file

Process
1. If prj exists, delete prj
2. create a new prj, then import "export prj name"+_ds_wp_prj.zip to new prj

In [1]:
import sys
import os
import yaml
import subprocess

In [2]:
# point to a local file with credentials. It is not synced to git.
CONFIG_FILE = "cpd484_jb_import.yaml"

In [3]:
# Load parameters from the YAML file
with open(CONFIG_FILE, 'r') as file:
    config = yaml.safe_load(file)

DSJOB_URL = config['url']
DSJOB_USER = config['user']
DSJOB_PWD = config['password']
EXPORT_PRJ_NAME = config['export_prj_name']
EXPORT_ZIP_NAME = EXPORT_PRJ_NAME+"_ds_wp_prj.zip"
IMPORT_PRJ_NAME = config['import_prj_name']

In [4]:
# Enable dsjob
%env CPDCTL_ENABLE_DSJOB=true
%env CPDCTL_ENABLE_DATASTAGE=true
%env CPDCTL_ENABLE_VOLUMES=1
!echo $CPDCTL_ENABLE_DSJOB
!echo $CPDCTL_ENABLE_DATASTAGE
!echo $CPDCTL_ENABLE_VOLUMES

env: CPDCTL_ENABLE_DSJOB=true
env: CPDCTL_ENABLE_DATASTAGE=true
env: CPDCTL_ENABLE_VOLUMES=1
true
true
1


In [5]:
# Configure cpdctl with the parameters
!cpdctl config user set CP4D-user --username "$DSJOB_USER" --password "$DSJOB_PWD"
!cpdctl config profile set CP4D-profile --url "$DSJOB_URL" --user CP4D-user
!cpdctl config profile use CP4D-profile

Switched to profile "CP4D-profile".


In [6]:
# list all projects
!cpdctl dsjob list-projects

...
Business Catalog Enrichment
GDPR Lineage
DataStage Import
test_python_pipeline
DQ Definition_Rules
Multicloud Data Integration L3 Tech Lab
Data Replication Lab
Auto Policy Risk
evaluate-an-ml-model
Data-Science-and-MLOps

Total: 10 Projects

Status code = 0


In [7]:
# check if import prj exists or not
#!cpdctl project list --name "$IMPORT_PRJ_NAME" --match exact

import subprocess

# Execute the command
command = f'cpdctl project list --name "{IMPORT_PRJ_NAME}" --match exact'
result = subprocess.run(command, shell=True, capture_output=True, text=True)

# Capture the return code and output
return_code = result.returncode
output = result.stdout

# Print the return code and output
print("Return Code:", return_code)
print("Output:", output)

# Check if the output contains "Nothing to show"
if "Nothing to show" in output:
    print("The project does not exist or there was an error.")

else:
    print("The project exists.")
    # if exist delete project
    !cpdctl dsjob delete-project --project "$IMPORT_PRJ_NAME"


Return Code: 0
Output: ...
ID                                     Name               Created                    Description                                  Type
83d06338-e4de-4697-8819-59262052f29f   DataStage Import   2024-05-30T06:20:39.553Z   This Project is created using dsjob plugin   cpd

The project exists.
...
{
    "StatusCode": 204,
    "Headers": {
        "Date": [
            "Thu, 30 May 2024 06:49:46 GMT"
        ],
        "Server": [
            "---"
        ],
        "Set-Cookie": [
            "2ac1df5a53d05af1ed3c0e46006c8757=b3bc268910522097895d67f7d3a9288f; path=/; HttpOnly; Secure; SameSite=Lax"
        ],
        "Strict-Transport-Security": [
            "max-age=31536000; includeSubDomains",
            "max-age=31536000; includeSubDomains"
        ],
        "X-Frame-Options": [
            "SAMEORIGIN"
        ]
    },
    "Result": null,
    "RawResult": null
}



In [8]:
# Create prj
!cpdctl dsjob create-project --name "$IMPORT_PRJ_NAME" 

fd46b0f1-8489-4393-bdae-559bb24ea53f


In [12]:
# missing 2 connections, one is to local DB2 connection, one is a storage volume connection
!cpdctl dsjob import-zip --project "$IMPORT_PRJ_NAME" --file-name "$EXPORT_ZIP_NAME" --conflict-resolution replace --wait 200 

...
2024-05-30 16:57:38: Waiting until import finishes, import id: 50e61d80-6ce6-4a63-b9c6-7bf03d1854f1
2024-05-30 16:57:49: Project import status: completed,  total: 3, completed: 1, failed: 0, skipped: 2.
Information:
	Connection: Data Virtualization,	  New connection is exactly the same as an existing connection, resource is not updated.

	Connection: Data Warehouse,	  New connection is exactly the same as an existing connection, resource is not updated.


Status code =  0
